# 10.3 실습: 의미 검색 구현하기
- Semantic Search
- 단순히 키워드 매칭을 통한 검색이 아니라, 밀집 임베딩을 이용해 문장이나 문서의 의미를 고려한 검색을 수행하는 것

## 10.3.1 의미검색 구현하기

In [1]:
# 예제 10.8 실습에 사용할 모델과 데이터셋 불러오기

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
klue_mrc_dataset

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 17554
})

In [2]:
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
sentence_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [3]:
# 예제 10.9 실습 데이터에서 1,000개만 선택하고 문장 임베딩으로 변환

train_data_set = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)['train']
train_data_set

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 1000
})

In [4]:
embeddings = sentence_model.encode(list(train_data_set['context']))
embeddings.shape

(1000, 768)

In [5]:
# 예제 10.10 KNN 검색 인덱스를 생성하고 문장 임베딩 저장

import faiss

# 인덱스 만들기
index = faiss.IndexFlatL2(embeddings.shape[1])

# 인덱스에 임베딩 저장하기
index.add(embeddings)
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x12fc1d050> >

In [6]:
# 예제 10.11 의미 검색의 장점

query = '이번 연도에는 언제 비가 많이 올까?'
query_embeddings = sentence_model.encode([query])
distances, indices = index.search(query_embeddings, 3)  # query_embeddings와 가장 가까운 3개의 문장 검색

indices

array([[  0, 920, 921]])

In [7]:
for idx in indices[0]:
    print(idx, train_data_set['context'][int(idx)], '\n-------------------------------')

0 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다. 
-------------------------------
920 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그 중에서도 태평양먹장어(Eptatretus stoutii)와 구조가 비슷하다. 게다가 오리너구리의 눈에서는 같은 포유류에게서 찾아보기 힘든 이중 원추세포가 발견된다. 

크게 발달하지도 않았고 물 속에서도 시야 확보를 위하여 쓰이지 않는 눈이지만, 오리너구리의 조상뻘 되는 종들이 한때 시각에 보다 크게 의존했다는 증거는 많이

In [8]:
# 예제 10.12 의미 검색의 한계

query = klue_mrc_dataset[3]['question']  # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
    print(klue_mrc_dataset['context'][int(idx)][:50])

태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스


## 10.3.2 라마인덱스에서 Sentence-Transformers 모델 사용하기


In [9]:
# 예제 10.13 라마인덱스에서 Sentence-Transformers 임베딩 모델 활용

from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core import Document, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = None  # LLM을 사용하지 않으므로 None으로 설정
Settings.embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 로컬 모델 활용하기
# service_context = ServiceContext.from_defaults(embed_model="local")

text_list = klue_mrc_dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

index_llama = VectorStoreIndex.from_documents(documents)
index_llama

LLM is explicitly disabled. Using MockLLM.
